# Closed loop simulations

The objective in this notebook is to close the former strategy in Symuvia. 

In [64]:
import os

from ctypes import cdll, create_string_buffer, c_int, byref, c_bool, c_double

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

from xmltodict import parse
from xml.parsers.expat import ExpatError

from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Spectral8, Reds, Greens, Category20
from bokeh.models import HoverTool, CategoricalColorMapper, Label

output_notebook() 

import ipywidgets as widgets
from IPython.display import display

from symuviapy.symfunc import queueveh, getlead, getspace, getleaderspeed, updatelist, typedict, check_veh_creation
from symuviapy.contfunc import compute_control, format_open_loop, solve_tactical_problem, headway_reference


Loading BokehJS ...

#### 1. Setting Parameters

Traffic simulation parameters 

In [65]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

#### 2. Database engine configuration

Reading database files 

In [66]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

#### 3. Open dB Connection + Table `headway`

In [67]:
if os.path.isfile(engine_full_name):
    try:
        ltbstr = 'Loaded table in: '
        connection = engine.connect()        
        traj = Table('traj', metadata, autoload=True, autoload_with=engine)        
        closed = Table('closed', metadata, autoload=True, autoload_with=engine)
        headway = Table('headway', metadata, autoload=True, autoload_with=engine)
        control = Table('control', metadata, autoload=True, autoload_with=engine)
    except:
        ltbstr = 'Created table in: '
        traj = Table('traj', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))        
        closed = Table('closed', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))
        control = Table('control', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('ctr', Float()),
                 Column('nit', Integer())) 
        headway = Table('headway', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('gapt', Float()))
        metadata.create_all(engine)
        connection = engine.connect()
    finally: 
        print(ltbstr, engine)
                

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


#### 3. Load traffic library

Load Symuvia library: 

In [68]:
dir_path = os.getcwd()
lib_path_name = ('..','SymuviaModPos','Contents','Frameworks','libSymuVia.dylib')
full_name = os.path.join(dir_path,*lib_path_name)
symuvialib = cdll.LoadLibrary(full_name)     

#### 4. Load traffic network 

Load traffic network 

In [69]:
file_path = ('..', 'Network', 'Merge_Demand_HDV.xml')
file_name = os.path.join(dir_path, *file_path)

# Pointers
sRequest = create_string_buffer(100000)
bEnd = c_int()
bSecond = c_bool(True)
bForce = c_int(1)

#### 5. Widgets for evolution 

Helpful widgets to visualize progress in the simulation 

In [70]:
max_time = 120.0
progressSim = widgets.FloatProgress(
    value=0,
    min=0,
    max=max_time,
    step=0.1,
    description='Simulating:',
    bar_style='info',
    orientation='horizontal'
)
tiVal = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=max_time,
    step=0.1,
    description='Time step:',
    disabled=False
)

#### 6. Start closed loop

Within the closed loop at each time step the following activities are performed 

0. Retrieve sample
1. Format samples 
3. Check initial conditions/ Enable control
2. Allocate samples
4. Solve tactical strategy (solved once) 
3. Solve control for CAV 
4. Compute evolution with corresponding control 
5. Format data 
6. Write in dataBase 
7. Update Symuvia 
8. Go to 0.



In [71]:
def create_platoon(lVehDataFormat):
    """
        Create platoon of vehicles 
    """
    lPlatoon = {}
    roads = []
    try:
        for veh in lVehDataFormat: 
            key = veh['tron'] 
            if key in roads:
                if veh['type'] == 'CAV':            
                    lPlatoon[key].append(veh['id'])
            else: 
                roads.append(key)            
                lPlatoon[key] = [veh['id']] if veh['type']=='CAV' else []
    except TypeError:
        veh = lVehDataFormat
        lPlatoon = [veh['id']] if veh['type']=='CAV' else []
        
    return lPlatoon

def format_data_controller(lVehDataFormat, lPlatoon):
    """
        Format data at time t for the controller, and
        boundary data
    """
    roads = [] 
    control_data = defaultdict(list)
    bound_data = {}
    bBound = False
    for veh in lVehDataFormat:         
        key = veh['id'] 
        veh_data = (float(veh['ti']),
                    veh['id'],
                    veh['type'],
                    veh['tron'],
                    veh['voie'],
                    veh['dst'],
                    veh['abs'],
                    veh['vit'],
                    veh['ldr'],
                    veh['spc'],
                    veh['vld'],
                   )

        for road, platoon in lPlatoon.items():
            if key in platoon:
                control_data[road].append(veh_data)
                break
            bBound = True
        key_rd = veh['tron']
        if key_rd in roads:            
            bound_data[key_rd].append(veh_data) 
            bBound = False
        elif bBound: 
            roads.append(key_rd)
            bound_data[key_rd] = [veh_data]
            bBound = False
        else:
            continue
            
    control_values = list(dict(control_data).values())
    bound_values = list(bound_data.values())
    return control_values, bound_values 

In [72]:
# %load symuviapy/contfunc.py
import numpy as np
import pandas as pd

from symuviapy.symfunc import updatelist

DT = 0.1  # Sample time

KC = 0.16  # CAV max density
KH = 0.0896  # HDV max density
VF = 25.0  # Speed free flow
W = 6.25  # Congestion speed
E = 25.0*0.3  # Speed drop for relaxation

GCAV = 1/(KC*W)  # Time headway CAV
GHDV = 1/(KH*W)  # Time headway HDV
SCAV = VF/(KC*W)+1/KC  # Desired space headway CAV
SHDV = VF/(KH*W)+1/KH  # Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5  # Max. Acceleration
U_MIN = -1.5  # Min. Acceleration

# Imposed leadership
dveh_ldr = {0: 0, 1: 0, 2: 1, 3: 2, 5: 3, 6: 5, 8: 6, 9: 8}
dveh_idx = {0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 8: 6, 9: 7}


def reversedEnumerate(*args):
    """ Inverse enumeration iterator"""
    revArg = [np.flip(x, axis=0) for x in args]
    return zip(range(len(args[0])-1, -1, -1), *revArg)


def find_idx_ldr(results, lPlatoon = None):
    #     """ From dbQuery finds idx or leader for CAVs"""
    # Frozen network (A-priori)

    if lPlatoon is not None:
        key = lPlatoon
        idx = list(range(len(key)))
        ldr = [key[0]]+ key [:-1]
        dveh_ldr = dict(zip(key,ldr))
        dveh_idx = dict(zip(key,idx))

    ldrl = [dveh_ldr[x[1]] for x in results if x[2] == 'CAV']
    idx_ldr = [dveh_idx[x] for x in ldrl]

    return idx_ldr, ldrl


def initial_setup_mpc(results, h_ref):
    """ Initialize variables for controller
    """

    TGref = h_ref  # format_reference(h_ref)
    h = TGref.shape[0]

    n_CAV = len([ty[2] for ty in results if ty[2] == 'CAV'])
    dCAVu = [h, n_CAV]
    # print(f'Dimensions control: {dCAVu}')

    Sref = np.zeros(dCAVu)

    S = np.zeros(dCAVu)
    V = np.zeros(dCAVu)
    DV = np.zeros(dCAVu)
    Lv = np.zeros(dCAVu)
    Ls = np.zeros(dCAVu)
    return (Sref, TGref, S, V, DV, Ls, Lv)


def format_reference(h_ref):
    """ Convert query from a reference into a 
        numpy array
    """

    # Rearrange
    refDf = pd.DataFrame(h_ref, columns=['ti', 'id', 'gapt'])
    # Pivot to pass vehicles as columns
    refMat = pd.pivot_table(refDf, index='ti', columns='id')['gapt']
    refMat = refMat.as_matrix()

    return refMat


def compute_control(results, h_ref, u_lead, lPlatoon = None):

    _, Tgref, S, V, DV, Ls, Lv = initial_setup_mpc(results, h_ref)

    
    # Static leadership
    ldr_pos, _ = find_idx_ldr(results, lPlatoon)

    S0 = [s[9] for s in results if s[2] == 'CAV']
    V0 = [v[7] for v in results if v[2] == 'CAV']
    DV0 = [dv[10]-dv[7] for dv in results if dv[2] == 'CAV']
    U_ext = Lv
    # U_ext[:,0] = u_lead # Head acceleration (external)

    # Initialize global variables
    S[0] = S0
    V[0] = V0
    DV[0] = DV0
    h = len(S)
    n = 0
    n_prev = 0

    # Parameters
    C1 = 0.1
    C2 = 1
    C3 = 0.5
    ALPHA = 0.01
    1
    EPS = 0.1
    error = 100

    bSuccess = 2
    N = 100001  # number of iterations
    step = iter(range(N))

    while (error > EPS) and (bSuccess > 0):
        try:
            next(step)
            U_star = -Lv/(2*C3)
            U_star = np.clip(U_star, U_MIN, U_MAX)

            DU = U_star[:, ldr_pos]-U_star[:] + U_ext

            # Forward evolution
            for i, u_s, du in zip(range(h), U_star, DU):
                if i < len(S)-1:
                    DV[i+1] = DV[i] + DT * du
                    S[i+1] = S[i] + DT * DV[i]
                    V[i+1] = V[i] + DT * u_s

            Sref = V * Tgref + 1/KC

            # Forward plots
            # plot_forward(Sref, Tgref, S, V, DV, U_star)

            ls = np.zeros(Ls.shape)
            lv = np.zeros(Lv.shape)

            # Backward evolution
            for i, s, v, dv, tg in reversedEnumerate(S, V, DV, Tgref):
                if i > 0:
                    sref = v * tg + 1/KC
                    lv[i-1] = lv[i] + DT * \
                        (-2 * C1 * (s-sref) * tg - C2 * dv - ls[i])
                    ls[i-1] = ls[i] + DT * (2 * C1 * (s-sref))

            # Update
            Ls = (1 - ALPHA) * Ls + ALPHA * ls
            Lv = (1 - ALPHA) * Lv + ALPHA * lv

            # Backwards plots
            # plot_backwards(ls, lv, Ls, Lv)

            error = np.linalg.norm(Ls - ls) + np.linalg.norm(Lv-lv)
            # print(f'Iteration: {n}, Error: {error}')

            # Routine for changing convergence parameter

            if error > 10e5:
                raise AssertionError('Algorithm does not converge ')
            if n >= 500:
                ALPHA = max(ALPHA - 0.01, 0.01)
                #print(f'Reaching {n} iterations: Reducing alpha: {ALPHA}')
                #print(f'Error before update {error}')
                if n > 10000:
                    raise AssertionError(
                        'Maximum iterations reached by the algorithm')
                n_prev = n + n_prev
                n = 0
            if error <= EPS:
                bSuccess = 0

            n += 1

        except StopIteration:
            print('Stop by iteration')
            print('Last simulation step at iteration: {}'.format(n+n_prev))
            bSuccess = 0

    n = n + n_prev
    return (S, V, DV, U_star, DU, n, Sref)


def determine_lane_change(CAVabsP):
    """ Returns the tuple (tron, voie) for a 
        CAV vehicle based on positions updates.
    """

    # Works only in current network

    CAVtron = []
    CAVvoie = []

    for abs_x in CAVabsP:
        if (abs_x <= 0):
            CAVtron.append('In_main')
            CAVvoie.append(1)
        elif (abs_x > 0) and (abs_x <= 1000.0):
            CAVtron.append('Merge_zone')
            CAVvoie.append(2)
        else:
            CAVtron.append('Out_main')
            CAVvoie.append(1)
    return CAVtron, CAVvoie


def update_state(S, V, DV, U_star, DU, n, results_closed, lPlatoon = None):
    """ Updates the state and computes closed loop updates
    """

    # NOTE: To be taken into account. Closed loop simulations
    # run without Symuvia. Requires implementation of the connection
    # NO LANE CHANGE MODEL IMPLENTED FOR HDV

    # Forward evolution
    DVp = DV[0] + DT * DU[0]
    Sp = S[0] + DT * DV[0]
    Vp = V[0] + DT * U_star[0]

    # t_i, id, type, from (results_closed):
    # Keep the same along the simulation
    CAVti = [x[0] + DT for x in results_closed if x[2] == 'CAV']
    CAVti = [float(np.round(x, 1)) for x in CAVti]
    CAVid = [x[1] for x in results_closed if x[2] == 'CAV']
    CAVtype = [x[2] for x in results_closed if x[2] == 'CAV']

    # Updates

    # Postition query
    CAVdst = [x[5] for x in results_closed if x[2] == 'CAV']
    CAVabs = [x[6] for x in results_closed if x[2] == 'CAV']

    # Updates from closed loop
    CAVdstP = [x + DT * v for (x, v) in zip(CAVdst, Vp)]  # or Vp?
    CAVabsP = [x + DT * v for (x, v) in zip(CAVabs, Vp)]  # or Vp?

    # Lane change
    CAVtron = [x[3] for x in results_closed if x[2] == 'CAV']
    CAVvoie = [x[4] for x in results_closed if x[2] == 'CAV']

    # State updates
    ldr_pos, ldr_list = find_idx_ldr(results_closed, lPlatoon)
    CAVvitP = Vp
    CAVldr = ldr_list
    CAVldr = [int(x) for x in ldr_list]
    CAVspcP = Sp
    CAVvldP = DVp + Vp

    
    # Render CAV results for dB
    CAViter = zip(CAVti, CAVid, CAVtype, CAVtron, CAVvoie,
                  CAVdstP, CAVabsP, CAVvitP, CAVldr,
                  CAVspcP, CAVvldP)

    keys = ('ti', 'id', 'type', 'tron', 'voie',
            'dst', 'abs', 'vit', 'ldr',
            'spc', 'vld')

    keysU = ('ti', 'id', 'type', 'tron', 'voie',
             'ctr', 'nit')

    lVehTrajCL = []
    for i in CAViter:
        lVehTrajCL.append(dict(zip(keys, i)))

    n_list = [n]*U_star.shape[0]

    CAViter = zip(CAVti, CAVid, CAVtype, CAVtron,
                  CAVvoie, U_star[0], n_list)

    lVehUCL = []
    for i in CAViter:
        lVehUCL.append(dict(zip(keysU, i)))

    return lVehTrajCL, lVehUCL


def format_open_loop(results):
    """ Aux function 
        To write in the closed loop database 
        results from open loop without treatment
        (No control applied)
        Homogenizes results in terms of content
    """

    keys = ('ti', 'id', 'type', 'tron', 'voie',
            'dst', 'abs', 'vit', 'ldr',
            'spc', 'vld')
    keysU = ('ti', 'id', 'type', 'tron', 'voie',
             'ctr', 'nit')

    lVehTrajOL = []
    lVehUOL = []

    for i in results:
        print(len(results))
        lVehTrajOL.append(dict(zip(keys, i)))
        ti, vid, ty, tr, vo, _, _, _, _, _, _ = i
        u_tup = (ti, vid, ty, tr, vo, 0, 0)
        lVehUOL.append(dict(zip(keysU, u_tup)))

    return lVehTrajOL, lVehUOL


def find_projection(gi, VF, gm, W):
    """ Find the projection of point gi at speed VF
        over a point gm at speed -W
    """
    Xm, Tm = gm
    x, t = gi
    M1 = np.array([[1, W], [1, -VF]])
    b = np.array([Xm + W * Tm, x - VF * t])
    pg = np.linalg.solve(M1, b)
    return pg


def find_anticipation_time(veh, pg_eq):
    # Assuming eq.
    T_x = pg_eq[1] - veh['gm'][1]
    T_0 = veh['pg'][1]- veh['gm'][1]

    X_m = veh['gm'][0]

    t_i = veh['curr'][1]
    x_i = veh['curr'][0]

    T1 = t_i + (X_m-x_i) / VF

    T2 = (VF + W) * (1 / VF - 1 / E) * (T_x - T_0)

    T3 = E / 2 * (1 / U_MAX - 1 / U_MIN)

    T4 = (VF + W) * (T_x - T_0) / E

    T_a = T3 + T4

    T_y = T1 + T2 - T3

    print(f'T0: {T_0}, Tf: {T_x} Ta: {T_a}, Ty: {T_y}')   
    
    return T_a, T_y


def solve_tactical_problem(lVehDataFormat):
    """
        Create a dictionary indicating the trigger time as a key     
    """

    SAFETY = 0  # Put on 0 for flow maximization

    # Find Xm, Tm
    lArrivalTimes = [(-x['abs']/x['vit'], x['id'],
                      0.0,
                      float(x['ti'])-x['abs']/x['vit']) for x in lVehDataFormat]
    vehLeader = min(lArrivalTimes, key=lambda t: t[0])
    gm = (vehLeader[2], vehLeader[3])

    lProj = []

    for veh in lVehDataFormat:
        gi = (veh['abs'], float(veh['ti']))
        pg = find_projection(gi, veh['vit'], gm, W)
        bBoundary = True if veh['id'] == 0 or veh['type'] == 'HDV' else False
        lProj.append((pg,
                      veh['id'],
                      bBoundary,
                      dveh_twy[veh['type']],
                      dveh_dwy[veh['type']],
                      veh['type'],
                      (veh['abs'],float(veh['ti'])),
                      gm,
                      veh['ldr'],
                     ))

    keys = ('pg', 'id', 'bound', 'tau', 'd', 'type', 'curr','gm','ldr')
    lProj = [dict(zip(keys, x)) for x in lProj]

    # Natural order
    lProjSort = sorted(lProj, key=lambda t: t['pg'][1])

    # Find only boundaries
    lBound = [x for x in lProjSort if x['bound']]

    # CAV to allocate
    lVehAlloc = [x for x in lProjSort if not x['bound']]

    if len(lBound) > 1:
        # Multiple boundaries

        lBoundHead = lBound[0:-1]
        lBoundTail = lBound[1:]
        deltaT = [(x['pg'][1]-y['pg'][1])
                  for x, y in zip(lBoundTail, lBoundHead)]

        # Veh to allocate
        lNumVehAlloc = []
        for delta, lead in zip(deltaT, lBoundHead):
            tau = lead['tau']
            nveh = max((delta-(tau+SAFETY*GHDV))//GCAV+1, 0.0)
            lNumVehAlloc.append(nveh)

        nVehLast = len(lVehAlloc)-sum(lNumVehAlloc)
        lNumVehAlloc.append(nVehLast)

    else:
        # Single Boundary
        lNumVehAlloc = [len(lVehAlloc)]

    # Find order such that matches allocation
    newProjSort = []
    veh2Alloc = iter(lVehAlloc)
    addVeh = 0
    last_cav = []
    
    ## ---------------------------------------------Figure plot 
    p = figure(title = "Tactical", 
           width=800, 
           height=800, 
           x_range=(40, 55),
           y_range=(-60, 10),
          tools = ['lasso_select', 'wheel_zoom, reset'])

    p.xaxis.axis_label = 'Time [s]'
    p.yaxis.axis_label = 'Position [m]'

    q = figure(title = "Tactical (Before)", 
           width=800, 
           height=800, 
           x_range=(40, 55),
           y_range=(-60, 10),    
          tools = ['lasso_select', 'wheel_zoom, reset'])

    q.xaxis.axis_label = 'Time [s]'
    q.yaxis.axis_label = 'Position [m]'
    
    ## ---------------------------------------------Figure plot 

    for veh, number in zip(lBound, lNumVehAlloc):
        cap = number

        # Computation equilibria (Boundary)
        pg_eq = veh['pg']
        d_tau = 0.0
        arr_t = find_projection(pg_eq, VF, gm, 0)[1]
        t_ant, t_yld = find_anticipation_time(veh, pg_eq)

        # Updates for follower
        shift_x = veh['d']
        shift_t = veh['tau']

        # Storage
        updt_dict = {'pg_eq': pg_eq,
                     'd_tau': d_tau,
                     'arr_t': arr_t,
                     'tau_0': veh['tau'],
                     'tau_f': veh['tau']+d_tau,
                     't_ant': t_ant,
                     't_yld': t_yld,
                     }
        veh = updatelist(veh, [updt_dict])
        last_cav = veh if veh['type']=='CAV' else last_cav
        newProjSort.append(veh)
        addVeh += 1    
        
        p.circle(veh['pg_eq'][1], veh['pg_eq'][0], color = 'blue', size = 4)
        labels = Label(x=veh['pg_eq'][1], y=veh['pg_eq'][0],
             text= str(veh['id']))
        p.add_layout(labels)        
        q.circle(veh['pg'][1], veh['pg'][0], color = 'lightskyblue', size = 4)
        labels = Label(x=veh['pg'][1]+0.5, y=veh['pg'][0]+0.5,
             text= str(veh['id']))           
        q.add_layout(labels)
        
        while cap > 0:
            cav = next(veh2Alloc)

            pg_ref = veh['pg'] if cap == number else newProjSort[addVeh-1]['pg_eq']                        

            # Computation yielding 
            pg_eq = np.array([pg_ref[0] - shift_x, pg_ref[1] + shift_t])  
            arr_t = find_projection(pg_eq, VF, gm, 0)[1]
            t_ant, t_yld = find_anticipation_time(cav, pg_eq)         
#             print(f'Tarr: {arr_t}')
            
            # Compute ref controller 
            cavldr = [x for x in newProjSort if x['id']==cav['ldr']]
            if not cavldr:
                cavldr_eq = cav['pg']
                cavldr_pg = cav['pg']
                cavldr_gm = cav['gm']
                offset = cav['tau']
            else:
                cavldr_eq = cavldr[0]['pg_eq']
                cavldr_pg = cavldr[0]['pg']
                cavldr_gm = cavldr[0]['gm']
                offset = 0

            Tx_l = cavldr_eq[1]-cavldr_gm[1]
            T0_l = cavldr_pg[1]-cavldr_gm[1]            
            Tx = pg_eq[1]-cav['gm'][1]
            T0 = cav['pg'][1]-cav['gm'][1]
            tau_0 = T0 - T0_l+ offset
            tau_x = Tx - Tx_l+ offset
            print(f'id: {cav["id"]} Tau0: {tau_0} Taux: {tau_x}')
            
            # Updates for follower
            shift_x = cav['d']
            shift_t = cav['tau']   

            # Storage
            updt_dict = {'pg_eq': pg_eq,
                         'd_tau': tau_x-tau_0,
                         'arr_t': arr_t,
                         'tau_0': tau_0,
                         'tau_f': tau_x,
                         't_ant': t_ant,
                         't_yld': t_yld,
                         }
            cav = updatelist(cav, [updt_dict])
            last_cav = cav
            newProjSort.append(cav)
            addVeh += 1
            cap = cap - 1
            
            p.square(cav['pg_eq'][1]+0.5, cav['pg_eq'][0]+0.5, color = 'crimson', size = 6)
            labels = Label(x=cav['pg_eq'][1], y=cav['pg_eq'][0],
                 text= str(cav['id']))
            p.add_layout(labels)
            q.square(cav['pg'][1], cav['pg'][0], color = 'lightcoral', size = 6, name= str(cav['id']))
            labels = Label(x=cav['pg'][1]+0.5, y=cav['pg'][0]+0.5,
                 text= str(cav['id']))           
            q.add_layout(labels)
                              

    # Create event dictionary

    d_ev = {np.round(x['t_yld']-10, 1): (x['id'],
                                      x['tau_0'],
                                      x['tau_f'],
                                      x['t_ant'], 
                                      x['d_tau']) for x in lProj if x['type'] == 'CAV'}

    show(p)
    show(q)
    return d_ev


def headway_reference(gap_events):
    """ Determine the time signal for the reference 
        of the controller. 
    """

    ti = np.arange(800)*DT
    hr = []

    h_df = []

    for k, v in gap_events.items():
        ref = v[1] + (v[2]-v[1]) / (1 + np.exp(-8*(ti-(k+v[3]/2))/(v[3])))
        hr.append((ref, v[0]))
        df = pd.DataFrame(ti, columns=['ti'])
        df['id'] = v[0]
        df['tau'] = ref
        h_df.append(df)

    refDf = pd.concat(h_df)

    refDf = pd.pivot_table(refDf, index='ti', columns='id')['tau']

    return refDf


In [75]:
%%time

# Simulation steps
N = 1200
step = iter(range(N)) 

# Initialize simulation
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))

# Cleaning + insertion commands
stmt = delete(closed) 
connection.execute(stmt)
stmt = delete(control) 
connection.execute(stmt)
stmtwriteCL = insert(closed) 
stmtwriteUCL = insert(control)

# Widgets 
display(progressSim) 
display(tiVal)

# Constant + Initial values
bSuccess = 2
bEnableControl = False
bTacticalComputed = False 
bPrintTactical = False 
bFlagControl = True 

t = []
nVehInitial = {'In_main':8,
               'In_onramp':2,
               'Merge_zone':8,
               'Out_main':8,
              }

# Lane/ network improv 
dNetworkSegment = {'In_main': 'Merge_zone',
                   'In_onramp': 'Merge_zone',
                   'Merge_zone': 'Out_main',
                   'Out_main': 'Out_main',
                  }
dNetworkThresh = {'In_main': 1000,
                  'In_onramp': 903.5,
                  'Merge_zone': 100.0,
                  'Out_main': 1500,
                  }


while bSuccess>0:
    # 0. 
    bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
    
    try: 
        
        # 1. 
        dParsed = parse(sRequest.value.decode('UTF8'))        
        ti = dParsed['INST']['@val']
        
        if dParsed['INST']['TRAJS'] is None:
            
            # Empty network             
            pass 
        
        else:
            
            lVehDataRaw = dParsed['INST']['TRAJS']['TRAJ']                  
            lVehDataFormat = []
            
            try:
                # Single veh @ ti 
                
                lVehDataFormat = typedict(lVehDataRaw)
                lVehDataFormat['ti'] = ti
                dLeader = {lVehDataFormat['tron']: [lVehDataFormat['id']]}                
                lVehDataFormat['ldr'] = getlead(dLeader, lVehDataFormat)  
                
            except TypeError:
                
                # Multiple veh @ ti
                for i,  veh in enumerate(lVehDataRaw):
                    dVehData = typedict(veh)
                    dVehData['ti'] = ti
                    dLeader = queueveh(dLeader, dVehData)
                    dVehData['ldr'] = getlead(dLeader, dVehData)
                    lVehDataFormat.append(dVehData)
                    
            lSpacing = getspace(lVehDataFormat)
            lLeaderSpeed = getleaderspeed(lVehDataFormat)
            lVehDataFormat = updatelist(lVehDataFormat,lSpacing)
            lVehDataFormat = updatelist(lVehDataFormat,lLeaderSpeed)
            
            
            if bEnableControl and bFlagControl:
                if bTacticalComputed:
                    if bPrintTactical: 
                        print('Tactics computed {}'.format(ti))
                        bPrintTactical = False 
                                            
                    
                    # Allocate samples
                    control_data, bound_data = format_data_controller(lVehDataFormat, lVehCAVs)
                    
                    # Prediction horizon
                    tf = float(ti) + 5.0                    
                    bHorizon = (refDf.index >= float(ti)) & (refDf.index <= tf)                    
                    refFuture = refDf.loc[bHorizon,:]   
                    
                    for veh_data, id_platoon in zip(control_data, lVehCAVs.values()):            
                        
                        refPlatoon = refFuture[id_platoon].as_matrix()
                        S, V, DV, U_star, DU, n, Sref = compute_control(veh_data, refPlatoon, 0, id_platoon) 

                        lVehTrajCL, lVehU = update_state(S, V, DV, U_star, DU, n, veh_data, id_platoon)

#                         if n>1:
#                             print('{}'.format(ti))
#                             print('\n before: \n {}'.format(pd.DataFrame(veh_data)))
                        tdf = pd.DataFrame(lVehTrajCL)
                        tdf['sref'] = Sref[0]
                        print('\n Controlled: \n{}'.format(tdf[['ti', 'id', 'type', 'tron','voie', 'dst', 'abs','vit','ldr','spc','vld','sref']]))


                        for veh in lVehTrajCL:
                            # Determine dst according to relative position                            
                            if veh['abs'] < 0:
                                sTroncon = veh['tron'].encode('UTF8')
                                nVoie = c_int(veh['voie'])
                                dPos = c_double(veh['dst'])
                                Id = c_int(veh['id'])
                                symuvialib.SymDriveVehicleEx(Id, 
                                                             sTroncon, 
                                                             nVoie, 
                                                             dPos, 
                                                             bForce)    
                            else: 
                                bFlagControl = False        
                                
                        connection.execute(stmtwriteCL,lVehTrajCL)
                        connection.execute(stmtwriteUCL,lVehU) 
                        
                else: 
                    dTrigTau = solve_tactical_problem(lVehDataFormat)
                    refDf = headway_reference(dTrigTau)
                    bTacticalComputed = True 
                    bPrintTactical = True
            else:
                bEnableControl = check_veh_creation(lVehDataFormat, nVehInitial)
                lVehCAVs = create_platoon(lVehDataFormat)
            
           
            if not bFlagControl:   
                lVehTrajCL = lVehDataFormat
                try: 
                    lVehU = [{'ctr': 0,
                              'id': x['id'],
                              'nit': 0,
                              'ti': x['ti'],
                              'tron': x['tron'],
                              'type': x['type'],
                              'voie': x['voie']} for x in lVehDataFormat if x['type']=='CAV']
                except TypeError:
                    lVehTrajCL = [lVehDataFormat]
                    lVehU = [{'ctr': 0,
                              'id': lVehDataFormat['id'],
                              'nit': 0,
                              'ti': lVehDataFormat['ti'],
                              'tron': lVehDataFormat['tron'],
                              'type': lVehDataFormat['type'],
                              'voie': lVehDataFormat['voie'],
                             }]
                connection.execute(stmtwriteCL,lVehTrajCL)
                connection.execute(stmtwriteUCL,lVehU)                
  
            
        n = next(step)           
        t.append(ti)
        progressSim.value = ti
        tiVal.value = ti
    except StopIteration:
        print('Stop by iteration')
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
    except ExpatError:
        bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
        print('Return from Symuvia Empty: {}'.format(sRequest.value.decode('UTF8')))
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
        

FloatProgress(value=12.1, bar_style='info', description='Simulating:', max=120.0)

BoundedFloatText(value=12.1, description='Time step:', max=120.0, step=0.1)

T0: 0.0, Tf: 0.0 Ta: 5.0, Ty: 36.25
T0: 1.0, Tf: 1.0 Ta: 5.0, Ty: 37.5
id: 1 Tau0: 1.0 Taux: 1.0
T0: 2.0, Tf: 2.0 Ta: 5.0, Ty: 38.75
id: 2 Tau0: 1.0 Taux: 1.0
T0: 2.133759999999995, Tf: 2.133759999999995 Ta: 5.0, Ty: 38.917199999999994
T0: 3.0, Tf: 3.91947428571428 Ta: 8.831142857142833, Ty: 37.31820000000002
id: 3 Tau0: 1.0 Taux: 1.91947428571428
T0: 4.0, Tf: 4.91947428571428 Ta: 8.831142857142833, Ty: 38.56820000000002
id: 4 Tau0: 1.0 Taux: 1.0
T0: 5.320320000000002, Tf: 5.320320000000002 Ta: 5.0, Ty: 42.9004
T0: 5.0, Tf: 7.106034285714287 Ta: 13.775142857142862, Ty: 36.3574
id: 5 Tau0: 1.0 Taux: 2.186560000000007
T0: 6.0, Tf: 8.106034285714287 Ta: 13.775142857142862, Ty: 37.6074
id: 7 Tau0: 1.0 Taux: 1.0
T0: 7.0, Tf: 9.106034285714287 Ta: 13.775142857142862, Ty: 38.8574
id: 8 Tau0: 1.0 Taux: 1.0


Tactics computed 12.10

 Controlled: 
     ti  id type     tron  voie     dst     abs   vit  ldr    spc   vld  \
0  12.2   0  CAV  In_main     1  273.75 -726.25  25.0    0  31.25  25.0   
1  12.2   1  CAV  In_main     1  242.50 -757.50  25.0    0  31.25  25.0   
2  12.2   2  CAV  In_main     1  211.25 -788.75  25.0    1  31.25  25.0   
3  12.2   3  CAV  In_main     1  180.00 -820.00  25.0    2  31.25  25.0   
4  12.2   4  CAV  In_main     1  148.75 -851.25  25.0    3  31.25  25.0   
5  12.2   5  CAV  In_main     1  117.50 -882.50  25.0    4  31.25  25.0   
6  12.2   7  CAV  In_main     1   86.25 -913.75  25.0    5  31.25  25.0   
7  12.2   8  CAV  In_main     1   55.00 -945.00  25.0    7  31.25  25.0   

        sref  
0  31.250000  
1  31.250000  
2  31.250000  
3  31.250000  
4  31.250000  
5  31.250134  
6  31.250000  
7  31.250000  

 Controlled: 
     ti  id type     tron  voie     dst     abs   vit  ldr    spc   vld  \
0  12.3   0  CAV  In_main     1  276.25 -723.75  25.0    0  3

In [17]:
refdBDf = refDf.reset_index()
refdBDf = pd.melt(refdBDf, id_vars = 'ti')
refdBDf.columns = ['ti','id','gapt']
refdBDf.to_sql(name='headway', con = engine, if_exists='replace', index=False)
refdBDf.head()

,ti,id,gapt
0,0.0,0,1.0
1,0.1,0,1.0
2,0.2,0,1.0
3,0.3,0,1.0
4,0.4,0,1.0


#### Reading from dB

Reading `headway` table to check reference headway times 

In [18]:
# Bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Spectral8, Reds, Greens, Category20
from bokeh.models import HoverTool, CategoricalColorMapper

output_notebook() 

Loading BokehJS ...

In [19]:
stmt = select([closed])
results = connection.execute(stmt).fetchall()
stmt = select([traj])
resultsOL = connection.execute(stmt).fetchall()
column_names = closed.columns.keys()
trajCLDf = pd.DataFrame(results, columns = column_names)
column_names = traj.columns.keys()
trajOLDf = pd.DataFrame(resultsOL, columns = column_names)

In [20]:
trajHDVDf = trajOLDf[trajOLDf['type']=='HDV']

trajCLDf['id'] = trajCLDf['id'].astype('str')
trajHDVDf['id'] = trajHDVDf['id'].astype('str')

sourceCL = ColumnDataSource(trajCLDf)
sourceHDV = ColumnDataSource(trajHDVDf)

CAV_list = trajCLDf.id.unique().tolist()
HDV_list = trajHDVDf.id.unique().tolist()

CAV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 ==0 ]
HDV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0 ]

color_mapper_CAV = CategoricalColorMapper(factors=CAV_list, palette=CAV_colors)
color_mapper_HDV = CategoricalColorMapper(factors=HDV_list, palette=HDV_colors)

colorCAV = dict(field='id', transform=color_mapper_CAV)
colorHDV = dict(field='id', transform=color_mapper_HDV)

hover = HoverTool(tooltips=[('t','@ti'),('x','@abs'),('type','@type'), ('id','@id')])
hoverRef = HoverTool(tooltips=[('t','@ti'), ('id','@id'), ('tau','@tau')])
hoverCtr = HoverTool(tooltips=[('t','@ti'),('u','@ctr'), ('id','@id')])

/Users/ladino/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ladino/anaconda3/lib/python3.6/site-packages/bokeh/models/mappers.py:82: UserWarning: Palette length does not match number of factors. ['6', '9'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


In [21]:
stmt = select([closed])
results = connection.execute(stmt).fetchall()
stmt = select([traj])
resultsOL = connection.execute(stmt).fetchall()
column_names = closed.columns.keys()
trajCLDf = pd.DataFrame(results, columns = column_names)
column_names = traj.columns.keys()
trajOLDf = pd.DataFrame(resultsOL, columns = column_names)
stmt = select([headway])
results = connection.execute(stmt).fetchall()
column_names = headway.columns.keys()
headDf = pd.DataFrame(results, columns = column_names)

In [22]:
trajHDVDf = trajOLDf[trajOLDf['type']=='HDV']

trajCLDf['id'] = trajCLDf['id'].astype('str')
trajHDVDf['id'] = trajHDVDf['id'].astype('str')

sourceCL = ColumnDataSource(trajCLDf)
sourceHDV = ColumnDataSource(trajHDVDf)

CAV_list = trajCLDf.id.unique().tolist()
HDV_list = trajHDVDf.id.unique().tolist()

CAV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 ==0 ]
HDV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0 ]

color_mapper_CAV = CategoricalColorMapper(factors=CAV_list, palette=CAV_colors)
color_mapper_HDV = CategoricalColorMapper(factors=HDV_list, palette=HDV_colors)

colorCAV = dict(field='id', transform=color_mapper_CAV)
colorHDV = dict(field='id', transform=color_mapper_HDV)

hover = HoverTool(tooltips=[('t','@ti'),('x','@abs'),('type','@type'), ('id','@id')])
hoverRef = HoverTool(tooltips=[('t','@ti'), ('id','@id'), ('tau','@tau')])
hoverCtr = HoverTool(tooltips=[('t','@ti'),('u','@ctr'), ('id','@id')])

/Users/ladino/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ladino/anaconda3/lib/python3.6/site-packages/bokeh/models/mappers.py:82: UserWarning: Palette length does not match number of factors. ['6', '9'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


<a id='space_headway'></a>
## Space headway

Head space under control 

In [23]:
p = figure(title = "Space headway", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom, reset'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Spacing [m]'

# Spacing 

p.circle('ti', 'spc', source=sourceCL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.add_tools(hover)
show(p)

#
#trajDf.plot(kind='scatter',x='ti',y ='spc', c = trajDf.tron.map(colormap), figsize = (7.5, 7.5), s = 2, grid = True);

<a id='time_headway'></a>
## Time headway 

Time headway under control 

In [24]:
# Control performance 

p = figure(title = "Time Gap", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Tau [s]'

trajCLDf['tau'] = (trajCLDf['spc']-1/KC).divide(trajCLDf['vit'])
headDf['id'] = headDf['id'].astype('str')
headDf['tau']=headDf['gapt']

REF_list = headDf.id.unique().tolist()

REF_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0]

color_mapper_REF = CategoricalColorMapper(factors=REF_list, palette=REF_colors)

sourceTau = ColumnDataSource(trajCLDf)
sourceRef = ColumnDataSource(headDf)

colorREF = dict(field='id', transform=color_mapper_REF)

p.circle('ti', 'tau', source=sourceTau, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.circle('ti', 'tau', source=sourceRef, color = colorREF, selection_color = colorREF, size = 3, alpha = 0.2, nonselection_alpha=0.1)
p.add_tools(hoverRef)
show(p)



/Users/ladino/anaconda3/lib/python3.6/site-packages/bokeh/models/mappers.py:82: UserWarning: Palette length does not match number of factors. ['8', '9'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


<a id='trajectories'></a>
## Trajectories 

Space time after control 

In [76]:
p = figure(title = "Trajectories", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom','box_zoom','reset'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'X [m]'

#p.circle('ti', 'abs', source=sourceHDV, color = colorHDV, selection_color = colorHDV, size = 1, nonselection_alpha=0.1)
p.circle('ti', 'abs', source=sourceCL, color = colorCAV, selection_color = colorCAV, size = 1, nonselection_alpha=0.1)
# p.circle('ti', 'abs', source=sourceHDV, color = 'salmon', size = 3)
# p.circle('ti', 'abs', source=sourceCL, color = 'deepskyblue', size = 3)
p.add_tools(hover)

show(p)
